# Script para web scraping Strava

>O script se baseia no uso de 2 bibliotecas, a ***Selenium*** e a ***Beatiful Soup***. Sendo a primeira responsável pela navegação nas páginas, isto é, desde login até movimentação da página, acessando hyperlinks e botões em java Script. Já a segunda é responsável pela cópia do código fonte, a partir da configuração de paramêtros, e retorno das chamadas. Em grandes linhas, buscamos a tabela onde os dados estão, e posteriormente extraimos a informação necessária.

*Necessário: Instalação das bibliotecas Selenium e BeautifulSoup, e também do webdriver do chrome para que o navegador seja acessado remotamente.

In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

1. Inicialização do navegador remotamente, e carregamento da página inicial do strava. O tempo de 5s é para o carregamento inicial.
2. Solicitação de login e senha. 
3. O campo usuário e senha é localizado na página através da biblioteca Selenium, e posteriormente é preenchido e clicado.
4. Substituir o campo ***url_evento*** pela url do evento do strava, ou seja, a corrida.

## Alterar campos

In [ ]:
email1=input(prompt="Digite seu email strava:")
senha1=input(prompt="Digite sua senha strava:")
email2=input(prompt="Digite seu email strava:")
senha2=input(prompt="Digite sua senha strava:")

## Alterar campos

In [ ]:
driver = webdriver.Chrome()
driver.get('https://www.strava.com/login')
time.sleep(5)
url_evento = 'https://www.strava.com/segments/17494434' #carrega um segmento qualquer para pegar o elemento de login/senha
import_strava="BASE ESTUDO - CLEAN.csv"
export = "BASE ESTUDO - CLEAN COVID - STRAVA.csv"
ultima_pagina = 1


A identificação dos campos ***usuário*** e ***senha*** é feito através da funcionalidade "inspeção do código fonte".

In [ ]:
username = driver.find_element_by_id("email")
password = driver.find_element_by_id("password")
email=email1
senha=senha1

username.send_keys(email)
password.send_keys(senha)
driver.find_element_by_id("login-button").click() #encontrar o elemento botão
time.sleep(2)

driver.get(url_evento)
time.sleep(2)


> São utilizados 5 listas para o armazenamento das informações, sendo elas:
* results (n x 7) - ['classificacao','nome', 'ano','pace', 'fc', 'vam','tempo_liquido'] sem tratamento
* result_faixa_etaria (n x 1) - faixas etárias
* result_sexo (n x 1) - masculino / feminino
* data (n x 1) - ['classificacao','nome', 'ano','pace', 'fc', 'vam','tempo_liquido'] sem valores nulos
* link_atividades (n x 1) - link para ser direcionado aos detalhes do evento que o  atleta tenha participado

Foram criados ainda 3 dicionários, onde o índice é a posição para localização (xpath) do item:

Ex: ``element=driver.find_element_by_xpath('//*[@id="segment-results"]/div[2]/table/tbody/tr/td[4]/div/ul/li['+str(se)+']/a')``

O valor ***se*** e ***fe*** são iterados no laço ***for*** para os indices do sexo e faixa etária.
Futuramente o dicionário de ***classe_peso*** poderá ser utilizado, nesse momento os dados não foram captados devido a uma independência dos filtros de **faixa_etaria e classe_peso**. Poderá ser implementando um script complementar para o preenchimento desse campo e **merge** com os demais campos.

In [ ]:
results = []
result_faixa_etaria = []
result_sexo=[]
data = []
link_atividades = []
link_atleta = []

faixa_etaria={'2':'19 e abaixo','3':'20 a 24','4':'25 a 34','5':'35 a 44','6':'45 a 54','7':'55 a 64','8':'65 a 69',
              '9':'70 a 74','10':'75+'}
sexo={'2':'Homens','3':'Mulheres'}
classe_peso={'54 kg e abaixo','55 a 64 kg', '75 a 84 kg','85 a 95 kg','95 kg a 104 kg','105 kg a 114 kg','115 kg e acima'}

meses={'1':'01','2':'02','3':'03','4':'04','5':'05','6':'06','7':'07','8':'08','9':'09','10':'10','11':'11','12':'12'}

## Core
>O core está separado em 4 laços **for**, sendo o primeiro para seleção do filtro de **homens/mulheres**, o segundo para seleção do filtro de **faixa etária**, o terceiro para navegação entre as **paginas**, e o quarto para extração dos **dados**.

1. O script clica no sexo masculino
2. Seleciona a primeira faixa etária (19 e abaixo)
3. Mantém selecionado o sexo masculino
4. Armazena os dados da primeira página na variável **rows**
5. Utiliza-se um laço **for** para extração dos dados e preenchimento das listas (citadas anteriormente)
6. Clica-se na próxima página - caso não tenha uma próxima, sai do atual laço de extração.
7. Seleciona-se a próxima faixa etária, e retorno para o passo 3

Durante os laços a **url_evento** é carregada algumas vezes para organizar a navegação do chrome.

>Após a extração dos dados cria-se 4 dataframes, e em seguida suas colunas são concatenadas num único dataframe, e é gerado um arquivo csv.

In [ ]:
dados_atletas = pd.read_csv(import_strava)


In [ ]:
dados_atletas.shape

In [ ]:
basecompleta=pd.DataFrame(data=None,columns=['k201901','t201901','a201901','k201902','t201902','a201902','k201903','t201903','a201903','k201904','t201904','a201904','k201905','t201905','a201905','k201906','t201906','a201906','k201907','t201907','a201907','k201908','t201908','a201908','k201909','t201909','a201909','k201910','t201910','a201910','k201911','t201911','a201911','k201912','t201912','a201912','k202001','t202001','a202001','k202002','t202002','a202002','k202003','t202003','a202003','k202004','t202004','a202004','k202005','t202005','a202005','k202006','t202006','a202006','k202007','t202007','a202007','k202008','t202008','a202008','k202009','t202009','a202009','k202010','t202010','a202010','k202011','t202011','a202011','k202012','t202012','a202012'])
basecompleta=pd.concat([dados_atletas,basecompleta],axis=1)

In [ ]:
basecompleta

In [ ]:
#https://www.strava.com/athletes/15905354#interval_type?chart_type=miles&interval_type=month&interval=202005&year_offset=0
#for linha in range(1,dados_atletas.shape[0]):
x=1
aux=0
for linha in range(0,dados_atletas.shape[0]):
    
    
    try:
        data=[]
        results=[]
        for ano in range(2019,2021):
            for mes in range(1,13):
                driver.get(basecompleta['atleta'][linha]+'#interval_type?chart_type=miles&interval_type=month&interval='+str(ano)+meses[str(mes)]+'&year_offset=0')
                time.sleep(4)
                dados = driver.find_element_by_xpath('//*[@id="totals"]')
                html=dados.get_attribute('innerHTML')
                soup = BeautifulSoup(html, "html.parser") #estruturação do código na variável soup
                rows = soup.findAll("li")
                for row in rows:
                    a = [t.text.strip() for t in row.findAll("strong")]#[0:]
                    results.append(a)
        

        for i in range(10,82):
            basecompleta.loc[linha,basecompleta.columns[i]]=results[i-10]
        print('Atleta ',linha+1,' de ',dados_atletas.shape[0])
        basecompleta.to_csv(export,index=False)
#############################################################################################################        
        if aux%5==0:
            x = x * -1
            driver.close()
            if x==1:
                email=email1
                senha=senha1
                


            else:
                email=email2
                senha=senha2
                


            driver = webdriver.Chrome()#options=chrome_options)
            driver.get('https://www.strava.com/login')
            time.sleep(5)

            username = driver.find_element_by_id("email")
            password = driver.find_element_by_id("password")


            username.send_keys(email)
            password.send_keys(senha)
            driver.find_element_by_id("login-button").click() #encontrar o elemento botão
            time.sleep(2)
        
#############################################################################################################        
        aux=aux+1 
        
        
    except:
        basecompleta.to_csv(export,index=False)
        time.sleep(4)